# Quick start guide

This tutorial covers some basic usage patterms and best practices to help you get started with pyuppaal. Before you start, make sure you have installed pyuppaal following the [installation guide](TBD).

In [3]:
import pyuppaal as pyu

pyu.set_verifyta_path("D:\\uppaal\\bin-Windows\\verifyta.exe")
# pyu.set_verifyta_path("your/path/to/verifyta.exe") # Set the path to verifyta.exe

## A simple example

Pyuppaal is designed as a tool to solve partially observable systems and provide counter examples for safety properties. 

With this purpose, pyuppaal provides a simple interface to create and manipulate UPPAAL models. The following example shows how to create a simple model and how to check if a given property is satisfied.

We will use [pyuppaal_demo_PipeNet.xml](TBD) as an example model in the following usage examples.

![pipeNetModel](pipeNetModel.png)

In this model, there is a pipe net that has hidden paths between the Entry and three different Exits. The guard on the edge is the falling time for each path, e.g., if a ball goes through hidden_path1, it will take `T1_Min` to `T1_Max` seconds.
One day, two balls are put to the Entry at the global time (gclk) 0 and 1000, and observe the balls' at the global time 500 and 1550 from Exit1 and Exit2. 

In [4]:
model_path = "pyuppaal_demo_PipeNet.xml" # Use the PipeNet-model.

pipeNet = pyu.UModel(model_path) # Initialize the model from `xml` file.

pipeNet.set_queries('E<> Monitor0.Obs2') # Set query(s).

print("queries:", pipeNet.queries, '\n') # See available queries. 

trace = pipeNet.easy_verify() # Get one possible trace.

print("pattern:", trace.untime_pattern, '\n') # Print patterns

print("trace:", trace) # Print trace.

queries: ['E<> Monitor0.Obs2'] 

pattern: ['input_ball', 'hidden_path1', 'hidden_path3', 'exit1', 'input_ball', 'hidden_path1', 'hidden_path4', 'exit2'] 

trace: State [0]: ['PipeNet.Idle', 'Input._id16', 'Monitor0.Start', 'Observer._id19']
global_variables [0]: None
Clock_constraints [0]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - PipeNet.t ≤ 0; PipeNet.t - Input.input_clk ≤ 0; Input.input_clk - Observer.input_clk ≤ 0; Observer.input_clk - t(0) ≤ 0; ]
transitions [0]: input_ball: Input -> ['PipeNet']; Input._id16 -> Input._id17; PipeNet.Idle -> PipeNet.Cross1; 
-----------------------------------
State [1]: ['PipeNet.Cross1', 'Input._id17', 'Monitor0.Start', 'Observer._id19']
global_variables [1]: None
Clock_constraints [1]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - t(0) ≤ 500; gclk - PipeNet.t ≤ 0; PipeNet.t - Input.input_clk ≤ 0; Input.input_clk - Observer.i

## Add input and observation template

As mentioned above, pyuppaal is designed to solve partially observable systems. To do so, pyuppaal allows you to add input and observation templates to the model. 

In pyuppaal, TimedActions is used to descripe inputs and observations. TimedActions is a class with three lists, containing actions, lower bounds and upper bounds of an input or observation.



In [5]:
inputs = pyu.TimedActions(actions=['input_ball', 'input_ball'], lb=[0, 1000], ub=[0, 1000]) # Define an input.
observations = pyu.TimedActions(actions=['exit1', 'exit2'], lb=[500, 1550], ub=[500, 1550]) # Define an observation.

pipeNet.add_input_template(inputs) # Add input template.
pipeNet.add_observer_template(observations) # Add observation template.

traces = pipeNet.find_all_patterns() # Find all traces.
print(str(traces[-1])) # Peek the last trace.

State [0]: ['PipeNet.Idle', 'Input._id24', 'Monitor0.Start', 'Observer._id27', 'Monitor1._id32']
global_variables [0]: None
Clock_constraints [0]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - PipeNet.t ≤ 0; PipeNet.t - Input.input_clk ≤ 0; Input.input_clk - Observer.input_clk ≤ 0; Observer.input_clk - t(0) ≤ 0; ]
transitions [0]: input_ball: Input -> ['PipeNet', 'Monitor1']; Input._id24 -> Input._id25; PipeNet.Idle -> PipeNet.Cross1; Monitor1._id32 -> Monitor1._id33; 
-----------------------------------
State [1]: ['PipeNet.Cross1', 'Input._id25', 'Monitor0.Start', 'Observer._id27', 'Monitor1._id33']
global_variables [1]: None
Clock_constraints [1]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - t(0) ≤ 500; gclk - PipeNet.t ≤ 0; PipeNet.t - Input.input_clk ≤ 0; Input.input_clk - Observer.input_clk ≤ 0; Observer.input_clk - gclk ≤ 0; ]
transitions [1]: hidden_path1: Pi

The input and observation template added by pyuppaal. The cache file `*_pattern.xml` can be find in the same directory of the input model.

![pipeNetInput](pipeNetInput.png)

![pipeNetObserver](pipeNetObserver.png)

## Find all patterns

Except from providing one possible pattern for a given model, pyuppaal can also provide all the possible patterns by automatically generating Monitor templates to remove the existing patterns.

In [6]:
traces = pipeNet.find_all_patterns() # Find all possible traces.
patterns = [t.untime_pattern for t in traces]

print('patterns:', patterns, '\n') # Print all patterns.

print('num patterns:', len(patterns), '\n') # Print the length of traces.

print(str(traces[0])) # Print the first trace.

patterns: [['input_ball', 'hidden_path1', 'hidden_path3', 'exit1', 'input_ball', 'hidden_path1', 'hidden_path4', 'exit2'], ['input_ball', 'hidden_path1', 'hidden_path3', 'exit1', 'input_ball', 'hidden_path2', 'hidden_path5', 'exit2']] 

num patterns: 2 

State [0]: ['PipeNet.Idle', 'Input._id24', 'Monitor0.Start', 'Observer._id27']
global_variables [0]: None
Clock_constraints [0]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - PipeNet.t ≤ 0; PipeNet.t - Input.input_clk ≤ 0; Input.input_clk - Observer.input_clk ≤ 0; Observer.input_clk - t(0) ≤ 0; ]
transitions [0]: input_ball: Input -> ['PipeNet']; Input._id24 -> Input._id25; PipeNet.Idle -> PipeNet.Cross1; 
-----------------------------------
State [1]: ['PipeNet.Cross1', 'Input._id25', 'Monitor0.Start', 'Observer._id27']
global_variables [1]: None
Clock_constraints [1]: [t(0) - gclk ≤ 0; t(0) - PipeNet.t ≤ 0; t(0) - Input.input_clk ≤ 0; t(0) - Observer.input_clk ≤ 0; gclk - t(

Monitor template 1 add by pyuppaal. The cache file `*_pattern.xml` can be find in the same directory of the input model.

![pipeNetModelMonitor1](pipeNetMonitor1.png)

All the possible patterns are shown below. The red path suggests the possible pattern that the first ball may go through. It can be easily analyzed that there are two possible patterns, green and yellow paths, for the second ball to reach exit2. Pyuppaal can detect all the possible patterns that can explain the inputs and observations.

![pipeNetPatterns](pipeNetPatterns.png)